# Multi-Class Classification with DistilBERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install dataset
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Bert/data.csv')  
shuffled_df = df.sample(frac=1)
shuffled_df.to_csv("shuffled_data.csv", index=False)

In [ ]:
df = pd.read_csv('shuffled_data.csv')  

In [ ]:
df.head() 

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,D5jAKP4AEuOPiCRr4sbJWw,VaUgmU6O7KQK2WCfI_eboQ,iSHKYbQfB2q6sGnd7AD2_A,4,0,0,0,We've tried most of the Chinese restaurants in...,2015-01-04 19:47:44
1,9Vi_TzSq_t80KjIUoRqpeg,pXGJIcj8qHEUb3IhunQ-Qw,aIflpZXlmofFGelTlqgGEQ,3,0,0,0,Potstickers were good but the lo-mein my boyfr...,2018-08-25 04:43:00
2,ww6FLkCBWbzxRSRvUsLPrw,gAcz_T0JKVmXDWkFehJigQ,KP5OncF2jhT7_J1phHPPww,2,1,1,1,Had dinner here on a recent Friday evening. G...,2015-05-26 22:31:48
3,k4xVggYniF-HzMzGLHy4dQ,UQFE3BT1rsIYrcDvu_XVow,diwwt5QBU3-Thig9SVlsOA,3,0,0,0,Jersey Shore meets Reno. \r\n\r\nSurprisingly ...,2012-03-07 19:00:13
4,bYq7Mw3DDNTg-Jo1dhM8JA,Fa2-VD8cqcn98vqciVn3Gw,ixPTo6Hum7nNZ7A4VPhTXg,3,0,0,0,"Decent, quick, and relatively friendly service...",2013-05-11 23:56:42


In [ ]:
drop_columns = ['review_id','user_id','business_id','useful','funny','cool','date']
df = df.drop(drop_columns, axis=1)

In [ ]:
df.head() 

,stars,text
0,4,We've tried most of the Chinese restaurants in...
1,3,Potstickers were good but the lo-mein my boyfr...
2,2,Had dinner here on a recent Friday evening. G...
3,3,Jersey Shore meets Reno. \r\n\r\nSurprisingly ...
4,3,"Decent, quick, and relatively friendly service..."


In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
#Preapre dataset as input for Bert
class Ds(Dataset):
  def __init__(self, df, tokenizer):
    self.df=df
    self.tokenizer=tokenizer
  def __len__(self):
    return len(self.df)
  def __getitem__(self,index):
    item=self.df.iloc[index]
    text=item["text"]
    star=item["stars"]
    label=[0.,0.,0.,0.,0.]
    label[star-1]=1.
    tokens=self.tokenizer(text, return_tensors="pt", truncation=True, max_length=300, padding="max_length")
    return {"input_ids":torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask),"labels":label}
    

In [ ]:
#Tokenize data with DistilBertTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AutoConfig
config=AutoConfig.from_pretrained("distilbert-base-uncased")
config.num_labels=5
print(config.num_labels)
tokenzier=DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
print(config)
model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=5)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



5


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.b

In [ ]:
ds=Ds(df, tokenzier)

In [ ]:
ds[1]

{'input_ids': tensor([  101, 18911, 26348,  2545,  2020,  2204,  2021,  1996,  8840,  1011,
         24182,  2026,  6898,  2288,  4694,  1005,  1056,  2130, 12984,  1012,
          2027,  2020,  2074,  2524,  4895,  3597, 23461, 27130,  1998,  2070,
         23582,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
#Freeze Bert layers
for name, param in model.named_parameters():
  #print(name)
  if name.startswith("dis"):
    param.requires_grad=False
  else:
    print(name)
    param.requires_grad=True

pre_classifier.weight
pre_classifier.bias
classifier.weight
classifier.bias


In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy") 

In [ ]:
#Calcuate accuracy score 
def compute_metrics(eval_pred):

  logits,labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  labels = np.argmax(labels, axis=-1)
  return metric.compute(predictions=predictions, references=labels) 

'''
# Calculate precision, recall, f1, accuracy score 
import numpy as np
from datasets import load_metric

def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(labels, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

'''

'\n\n\nimport numpy as np\nfrom datasets import load_metric\n\ndef custom_metrics(eval_pred):\n    metric1 = load_metric("precision")\n    metric2 = load_metric("recall")\n    metric3 = load_metric("f1")\n    metric4 = load_metric("accuracy")\n    \n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=-1)\n    labels = np.argmax(labels, axis=-1)\n\n    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]\n    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]\n    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]\n    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]\n\n    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}\n\n'

In [ ]:
device="cuda"

In [ ]:
#Fine tune the model 
from transformers import TrainingArguments, Trainer
training_args=TrainingArguments(output_dir="/content/drive/MyDrive/Bert/",
                                evaluation_strategy="epoch",
                                num_train_epochs=50,
                                per_device_train_batch_size=64,
                                per_device_eval_batch_size=64,
                                save_steps=1e5,
                                weight_decay=0.01)  #weight_delay can be change 

#maybe add optimizer

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import DataCollatorWithPadding
train_ds,val_ds=torch.utils.data.random_split(ds, [9000, 1000])
data_collator=DataCollatorWithPadding(tokenizer=tokenzier)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    #compute_metrics=custom_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9000
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 7050
  Number of trainable parameters = 594437


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.475492,0.491000
2,No log,0.438635,0.506000
3,No log,0.411535,0.501000
4,0.461300,0.398109,0.498000
5,0.461300,0.389115,0.528000
6,0.461300,0.383135,0.531000
7,0.461300,0.380193,0.535000
8,0.393700,0.375721,0.529000
9,0.393700,0.373867,0.538000
10,0.393700,0.372174,0.535000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64
***** 

TrainOutput(global_step=7050, training_loss=0.36889116003158245, metrics={'train_runtime': 5812.9472, 'train_samples_per_second': 77.413, 'train_steps_per_second': 1.213, 'total_flos': 3.492979605e+16, 'train_loss': 0.36889116003158245, 'epoch': 50.0})

In [ ]:
from torch.utils.data import DataLoader
dl=DataLoader(val_ds)

In [ ]:
#Calculate accuracy per class/star
totalcount=[0,0,0,0,0]
totalcorrect=[0,0,0,0,0]
for i in dl:
  label=i["labels"]
  input_ids=i["input_ids"].to("cuda")
  attention_mask=i["attention_mask"].to("cuda")
  lo=model(input_ids, attention_mask).logits[0]
  lo=lo.cpu()
  lo=lo.detach().numpy()
  prediction = np.argmax(lo)
  label = np.argmax(label, axis=-1)
  totalcount[label]=totalcount[label]+1
  if label == prediction: 
    totalcorrect[label]=totalcorrect[label]+1
accuracy_based_class=torch.tensor(totalcorrect)/torch.tensor(totalcount)
print(accuracy_based_class)


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:43: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


tensor([0.7716, 0.3868, 0.4369, 0.3988, 0.7642])


In [ ]:
print(totalcount)
print(totalcorrect)
print(accuracy_based_class)

[197, 212, 206, 173, 212]
[152, 82, 90, 69, 162]
tensor([0.7716, 0.3868, 0.4369, 0.3988, 0.7642])


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


{'eval_loss': 0.35426533222198486,
 'eval_accuracy': 0.555,
 'eval_runtime': 11.5122,
 'eval_samples_per_second': 86.864,
 'eval_steps_per_second': 1.39,
 'epoch': 50.0}